In [35]:
from sklearn.preprocessing import LabelEncoder
from bio import AlignIO
from Bio.Align import MultipleSeqAlignment
from Bio.SeqRecord import SeqRecord
from Bio import SeqIO
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from keras_preprocessing import sequence
import sklearn
import numpy as np
import re

In [36]:
seq_length = 11
def string_to_array(my_string):
    my_string = my_string.lower()
    my_string = re.sub('[^acgt0]', 'z', my_string)
    my_array = np.array(list(my_string))
    return my_array

def readmaf(start, end, filename):
    count = 0
    oldlist =[]
    align = AlignIO.parse(filename, "maf")
    for multiple_alignment in align:
        count = count+1
        if count >start and count<=end:
            oldlist.append(multiple_alignment)
        elif count>end:
            break
    return oldlist

#oldlist stores MSAs
oldlist = readmaf(0, 1000000, "data/chr2.anc.maf")
print(oldlist[1][1])
print(dir(oldlist[1]))
print(oldlist[1][0].id)
#print(oldlist[2])

ID: panTro4.chr2A
Name: panTro4.chr2A
Number of features: 0
/start=4703
/size=1
/strand=1
/srcSize=113622374
Seq('a', SingleLetterAlphabet())
['__add__', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getitem__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__iter__', '__le__', '__len__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_alphabet', '_annotations', '_append', '_get_per_column_annotations', '_per_col_annotations', '_records', '_set_per_column_annotations', '_str_line', 'add_sequence', 'annotations', 'append', 'column_annotations', 'extend', 'format', 'get_alignment_length', 'sort']
hg38.chr2


In [37]:
print(len(oldlist))

1000000


In [38]:
#alignList stores alignments between direct ancestor and descendent
def getAlign(inputList, keyword):
    alignList = []
    zeros = ''
    for i in range(len(inputList)):
        # index for the dog
        indexDes = -1
        indexAnc = -1

        for j in range(len(inputList[i])):
            if len(inputList[i][j])>=10 and 'canFam3' in inputList[i][j].id  :
                indexDes = j
            elif len(inputList[i][j])>=10 and inputList[i][j].id == '_CMAOL': #'_RMPC' :
                indexAnc = j
        if indexDes!=-1 and indexAnc!=-1:
            alignment = MultipleSeqAlignment([
                    SeqRecord(inputList[i][indexDes].seq+zeros, id=inputList[i][indexDes].id),
                    SeqRecord(inputList[i][indexAnc].seq+zeros, id=inputList[i][indexAnc].id),
                    #SeqRecord(zeros+inputList[i][indexDes].seq, id=inputList[i][indexDes].id)
                ]
            )
            alignList.append(alignment)
    return alignList

def zeros(length):
    return "0"*length
    
FinalList = []    
anc = []
des = []
keyword = ["_HP","_RM","_FC","_CS","_BO","_PB","_TO","_VC", "_PP"]
alignList = getAlign(oldlist, keyword)
print(len(alignList))
print(alignList[126])
print(len(alignList[126][1].seq))
print(len(alignList[126][1]))
for i in range (100):
    print('printing new alignment -------------------------------------------\n')
    print(alignList[i])

#print(str(alignList[62][1]))
#print('The scikit-learn version is {}.'.format(sklearn.__version__))

357202
SingleLetterAlphabet() alignment with 2 rows and 20 columns
GATG---------------- canFam3.chr17
GATG---------------- _CMAOL
20
20
printing new alignment -------------------------------------------

SingleLetterAlphabet() alignment with 2 rows and 34 columns
AAGAGAG-AAACTAAAGCTGAGCCATAGGCTACA canFam3.chr17
AAGACAG-AAACTAAAGCTGAGCCATAGGCTACA _CMAOL
printing new alignment -------------------------------------------

SingleLetterAlphabet() alignment with 2 rows and 238 columns
A----------CTGACCTGAC----AGTCAGAGGAA-AGCAGGG...--- canFam3.chr17
A----------CTGATCTGACAATAAGTCAGGGGAA-AGAAGGG...--- _CMAOL
printing new alignment -------------------------------------------

SingleLetterAlphabet() alignment with 2 rows and 79 columns
------------------------------------------AG...TTT canFam3.chr17
------------------------------------------AG...TTT _CMAOL
printing new alignment -------------------------------------------

SingleLetterAlphabet() alignment with 2 rows and 14 columns
GCATTGTGTG---T

In [39]:
label_encoder = LabelEncoder()
label_encoder.fit(np.array(['a','c','g','t','0', 'z']))
encoder = OneHotEncoder(sparse=False, dtype=int, categories=[range(6)])
def one_hot_encoder(my_array, encoder):
    integer_encoded = label_encoder.transform(my_array)
    integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
    onehot_encoded = encoder.fit_transform(integer_encoded)
    #onehot_encoded = np.delete(onehot_encoded, -1, 1)
    return onehot_encoded, encoder

def alignBoth(alignList):
    zeros = '00000'
    h, w = len(alignList), 2;
    Matrix = [[0 for x in range(w)] for y in range(h)]
    for i in range(len(alignList)):
        for j in range(len(alignList[i])):
            Matrix[i][j] = one_hot_encoder(string_to_array(str(alignList[i][j].seq).upper()), encoder)
    return Matrix

# [Index of alignment][1 for Ancester, 0 for Descendant][Always 0]
one_hot_aligned = alignBoth(alignList)
#one_hot_aligned = one_hot_aligned[:][:][0]
one_hot_aligned = np.array(one_hot_aligned)
#print(one_hot_aligned.shape)
#ignorethis, encoder = one_hot_encoder(string_to_array(str(alignList[1][1].seq)),encoder)


print(type(one_hot_aligned))
print(len(one_hot_aligned[126][1]))
print(one_hot_aligned[126][1][0])
#print(encoder.inverse_transform(one_hot_aligned[126][1]))
print(type(one_hot_aligned[126][1]))
# A = [1 0 0 0], C = [0 1 0 0], G = [0 0 1 0], T = [0 0 0 1]
def decoder(array):
    result = ""
    size = len(array)
    for i in range(size):
        if array[i].tolist() == [1, 0, 0, 0, 0, 0]:
            result=result+"0" 
        elif array[i].tolist() == [0, 1, 0, 0, 0, 0]:
            result=result+"A"
        elif array[i].tolist() == [0, 0, 1, 0, 0, 0]:
            result=result+"C"
        elif array[i].tolist() == [0, 0, 0, 1, 0, 0]:
            result=result+"G"
        elif array[i].tolist() == [0, 0, 0, 0, 1, 0]:
            result=result+"T"
        elif array[i].tolist() == [0, 0, 0, 0, 0, 1]:
            result=result+"-"
    return result

def decoderY(array):
    result = ""    
    if array.tolist() == [1, 0, 0, 0, 0, 0]:
        result=result+"0" 
    elif array.tolist() == [0, 1, 0, 0, 0, 0]:
        result=result+"A"
    elif array.tolist() == [0, 0, 1, 0, 0, 0]:
        result=result+"C"
    elif array.tolist() == [0, 0, 0, 1, 0, 0]:
        result=result+"G"
    elif array.tolist() == [0, 0, 0, 0, 1, 0]:
        result=result+"T"
    elif array.tolist() == [0, 0, 0, 0, 0, 1]:
        result=result+"-"
    return result
    
print(decoder(one_hot_aligned[126][1][0]))
print(len(one_hot_aligned))

<class 'numpy.ndarray'>
2
[[0 0 0 1 0 0]
 [0 1 0 0 0 0]
 [0 0 0 0 1 0]
 [0 0 0 1 0 0]
 [0 0 0 0 0 1]
 [0 0 0 0 0 1]
 [0 0 0 0 0 1]
 [0 0 0 0 0 1]
 [0 0 0 0 0 1]
 [0 0 0 0 0 1]
 [0 0 0 0 0 1]
 [0 0 0 0 0 1]
 [0 0 0 0 0 1]
 [0 0 0 0 0 1]
 [0 0 0 0 0 1]
 [0 0 0 0 0 1]
 [0 0 0 0 0 1]
 [0 0 0 0 0 1]
 [0 0 0 0 0 1]
 [0 0 0 0 0 1]]
<class 'numpy.ndarray'>
GATG----------------
357202


In [40]:
def vectorize(char_setx, char_sety, sequence_length):
    dataX = []
    dataY = []
    dataX1 = []
    dataY1 = []
    result=[]
    char_setx, char_sety = mutation_with_gap(char_setx, char_sety)
    #print(type(char_sety[0]))
    for i in range(len(char_setx) - sequence_length):
        
        '''for i in char_sety[i: i + sequence_length-1]:
            y1.append(i)'''
        #x1 = char_setx1[i: i + sequence_length]
        x = char_setx[i:i + sequence_length]
        y = char_sety[i: i + sequence_length]
        y1 = [[1, 0, 0, 0, 0, 0]]
        temp = y[:-1]
        #temp = temp.tolist()
        for i in temp:
            y1.append(i)
        y1=np.array(y1)
        dataX.append(x)
        dataY.append(y)
        dataY1.append(y1)

    return dataX, dataY, dataY1

def tokenize(one_hot_aligned, sequence_length):
    X = []
    Y = []
    X1 = []
    Y1 = []
    for i in range(len(one_hot_aligned)):
        dataX, dataY, dataY1 = vectorize(one_hot_aligned[i][1][0], one_hot_aligned[i][0][0],  sequence_length)
        X.extend(dataX)
        Y.extend(dataY)
        #X1.extend(dataX1)
        Y1.extend(dataY1)
    return X, Y, Y1

def truncate(one_hot_aligned):
    X = []
    Y = []
    for i in range(len(one_hot_aligned)):
        X.append(one_hot_aligned[i][1][0])
        Y.append(one_hot_aligned[i][0][0])
    X = sequence.pad_sequences(X,
                                 maxlen=30,
                                 truncating='post',
                                 padding='post',
                                 value=0)
    Y = sequence.pad_sequences(Y,
                                 maxlen=30,
                                 truncating='post',
                                 padding='post',
                                 value=0)
    return X,Y

# function for checking if there are at least one mutation and there are no gaps.
def mutation_with_nogap(a, b):
    a = a.tolist()
    b = b.tolist()
    if a.count([0,0,0,0,0,1])>0 or b.count([0,0,0,0,0,1])>0:
        return False
    else:
        return True
    
def mutation_with_gap(a, b):
    a = a.tolist()
    b = b.tolist()
    a_new = []
    b_new = []
    for i, j in zip(a,b):
        if i == [0,0,0,0,0,1] and j == [0,0,0,0,0,1]:
            continue
        else:
            a_new.append(i)
            b_new.append(j)
    return a_new, b_new
        


def diffList(a, b):
    count = 0
    length = len(a)
    for i in range(length):
        if a[i] != b[i]:
            count = count+1
    return count

def deleteGap(one_hot_aligned):
    result = one_hot_aligned
    for i in range(len(one_hot_aligned)):
        if one_hot_aligned[i][1][0] == '-' and one_hot_aligned[i][0][0] == '-':
            result.pop(i)
    return result

# Used for non 0 padding
#one_hot_aligned = deleteGap(one_hot_aligned)
X, Y, Y1=tokenize(one_hot_aligned, seq_length)

# Used for 0 padding
#X, Y = truncate(one_hot_aligned)
X = np.array(X)
Y = np.array(Y)
#X1 = np.array(X1)
Y1 = np.array(Y1)

print(Y.shape)
print(len(X))
pt1 = 9600000
pt2 = 9650000
pt3 = 9700000
X_train = X[:pt1]
X_val = X[pt1:pt2]
X_test = X[pt2:pt3]
y_train = Y[:pt1]
y_val =  Y[pt1:pt2]
y_test = Y[pt2:pt3]
# X_train1 = X1[:pt1]
# X_val1 = X1[pt1:pt2]
# X_test1 = X1[pt2:pt3]
y_train1 = Y1[:pt1]
y_val1 =  Y1[pt1:pt2]
y_test1 = Y1[pt2:pt3]

# X_train = X[:20000]
# X_val = X[20000:23000]
# X_test = X[23000:26000]
# y_train = Y[:20000]
# y_val =  Y[20000:23000]
# y_test = Y[23000:26000]

np.save('prepData/X_train_camFam3_v3_chr2_size11', X_train)
np.save('prepData/X_val_camFam3_v3_chr2_size11', X_val)
np.save('prepData/X_test_camFam3_v3_chr2_size11', X_test)
np.save('prepData/y_train_camFam3_v3_chr2_size11', y_train)
np.save('prepData/y_val_camFam3_v3_chr2_size11', y_val)
np.save('prepData/y_test_camFam3_v3_chr2_size11', y_test)

np.save('prepData/y_train1_camFam3_v3_chr2_size11', y_train1)
np.save('prepData/y_val1_camFam3_v3_chr2_size11', y_val1)
np.save('prepData/y_test1_camFam3_v3_chr2_size11', y_test1)

# np.save('prepData/X_train1_camFam3_1mutOnly_v2', X_train1)
# np.save('prepData/X_val1_camFam3_1mutOnly_v2', X_val1)
# np.save('prepData/X_test1_camFam3_1mutOnly_v2', X_test1)


# np.save('prepData20/X_train_camFam3_1mutOnly', X_train)
# np.save('prepData20/X_val_camFam3_1mutOnly', X_val)
# np.save('prepData20/X_test_camFam3_1mutOnly', X_test)
# np.save('prepData20/y_train_camFam3_1mutOnly', y_train)
# np.save('prepData20/y_val_camFam3_1mutOnly', y_val)
# np.save('prepData20/y_test_camFam3_1mutOnly', y_test)

print(X_train.shape)
print(X_val.shape)
print(X_test.shape)
print(X_train[0])
print(y_train[0])
print(y_train1[0])


(9749947, 11, 6)
9749947
(9600000, 11, 6)
(50000, 11, 6)
(50000, 11, 6)
[[0 1 0 0 0 0]
 [0 1 0 0 0 0]
 [0 0 0 1 0 0]
 [0 1 0 0 0 0]
 [0 0 1 0 0 0]
 [0 1 0 0 0 0]
 [0 0 0 1 0 0]
 [0 1 0 0 0 0]
 [0 1 0 0 0 0]
 [0 1 0 0 0 0]
 [0 0 1 0 0 0]]
[[0 1 0 0 0 0]
 [0 1 0 0 0 0]
 [0 0 0 1 0 0]
 [0 1 0 0 0 0]
 [0 0 0 1 0 0]
 [0 1 0 0 0 0]
 [0 0 0 1 0 0]
 [0 1 0 0 0 0]
 [0 1 0 0 0 0]
 [0 1 0 0 0 0]
 [0 0 1 0 0 0]]
[[1 0 0 0 0 0]
 [0 1 0 0 0 0]
 [0 1 0 0 0 0]
 [0 0 0 1 0 0]
 [0 1 0 0 0 0]
 [0 0 0 1 0 0]
 [0 1 0 0 0 0]
 [0 0 0 1 0 0]
 [0 1 0 0 0 0]
 [0 1 0 0 0 0]
 [0 1 0 0 0 0]]


In [41]:
# import difflib
# sm=difflib.SequenceMatcher(None,X_test,X_train)
# sm2=difflib.SequenceMatcher(None,y_test,y_train)
for i in range(100):
    print(i, decoder(X[i]))

0 AAGACAGAAAC
1 AGACAGAAACT
2 GACAGAAACTA
3 ACAGAAACTAA
4 CAGAAACTAAA
5 AGAAACTAAAG
6 GAAACTAAAGC
7 AAACTAAAGCT
8 AACTAAAGCTG
9 ACTAAAGCTGA
10 CTAAAGCTGAG
11 TAAAGCTGAGC
12 AAAGCTGAGCC
13 AAGCTGAGCCA
14 AGCTGAGCCAT
15 GCTGAGCCATA
16 CTGAGCCATAG
17 TGAGCCATAGG
18 GAGCCATAGGC
19 AGCCATAGGCT
20 GCCATAGGCTA
21 CCATAGGCTAC
22 ACTGATCTGAC
23 CTGATCTGACA
24 TGATCTGACAA
25 GATCTGACAAT
26 ATCTGACAATA
27 TCTGACAATAA
28 CTGACAATAAG
29 TGACAATAAGT
30 GACAATAAGTC
31 ACAATAAGTCA
32 CAATAAGTCAG
33 AATAAGTCAGG
34 ATAAGTCAGGG
35 TAAGTCAGGGG
36 AAGTCAGGGGA
37 AGTCAGGGGAA
38 GTCAGGGGAAA
39 TCAGGGGAAAG
40 CAGGGGAAAGA
41 AGGGGAAAGAA
42 GGGGAAAGAAG
43 GGGAAAGAAGG
44 GGAAAGAAGGG
45 GAAAGAAGGGC
46 AAAGAAGGGCA
47 AAGAAGGGCAC
48 AGAAGGGCACA
49 GAAGGGCACAG
50 AAGGGCACAGA
51 AGGGCACAGAG
52 GGGCACAGAGA
53 GGCACAGAGAG
54 GCACAGAGAGA
55 CACAGAGAGAT
56 ACAGAGAGATT
57 CAGAGAGATTT
58 AGAGAGATTTG
59 GAGAGATTTGA
60 AGAGATTTGAA
61 GAGATTTGAAC
62 AGATTTGAACA
63 GATTTGAACAT
64 ATTTGAACATT
65 TTTGAACATTT
66 TTGAACATTTG
67 TG

In [42]:
for i in range(100):
    print(i, decoder(Y[i]))

0 AAGAGAGAAAC
1 AGAGAGAAACT
2 GAGAGAAACTA
3 AGAGAAACTAA
4 GAGAAACTAAA
5 AGAAACTAAAG
6 GAAACTAAAGC
7 AAACTAAAGCT
8 AACTAAAGCTG
9 ACTAAAGCTGA
10 CTAAAGCTGAG
11 TAAAGCTGAGC
12 AAAGCTGAGCC
13 AAGCTGAGCCA
14 AGCTGAGCCAT
15 GCTGAGCCATA
16 CTGAGCCATAG
17 TGAGCCATAGG
18 GAGCCATAGGC
19 AGCCATAGGCT
20 GCCATAGGCTA
21 CCATAGGCTAC
22 ACTGACCTGAC
23 CTGACCTGAC-
24 TGACCTGAC--
25 GACCTGAC---
26 ACCTGAC----
27 CCTGAC----A
28 CTGAC----AG
29 TGAC----AGT
30 GAC----AGTC
31 AC----AGTCA
32 C----AGTCAG
33 ----AGTCAGA
34 ---AGTCAGAG
35 --AGTCAGAGG
36 -AGTCAGAGGA
37 AGTCAGAGGAA
38 GTCAGAGGAAA
39 TCAGAGGAAAG
40 CAGAGGAAAGC
41 AGAGGAAAGCA
42 GAGGAAAGCAG
43 AGGAAAGCAGG
44 GGAAAGCAGGG
45 GAAAGCAGGGC
46 AAAGCAGGGCA
47 AAGCAGGGCAT
48 AGCAGGGCATA
49 GCAGGGCATAG
50 CAGGGCATAGA
51 AGGGCATAGAG
52 GGGCATAGAGA
53 GGCATAGAGAG
54 GCATAGAGAGG
55 CATAGAGAGGT
56 ATAGAGAGGTT
57 TAGAGAGGTTT
58 AGAGAGGTTTG
59 GAGAGGTTTGA
60 AGAGGTTTGAA
61 GAGGTTTGAAC
62 AGGTTTGAACA
63 GGTTTGAACAC
64 GTTTGAACACT
65 TTTGAACACTT
66 TTGAACACTTG
67 TG

In [43]:
for i in range(100):
    print(i, decoder(Y1[i]))

0 0AAGAGAGAAA
1 0AGAGAGAAAC
2 0GAGAGAAACT
3 0AGAGAAACTA
4 0GAGAAACTAA
5 0AGAAACTAAA
6 0GAAACTAAAG
7 0AAACTAAAGC
8 0AACTAAAGCT
9 0ACTAAAGCTG
10 0CTAAAGCTGA
11 0TAAAGCTGAG
12 0AAAGCTGAGC
13 0AAGCTGAGCC
14 0AGCTGAGCCA
15 0GCTGAGCCAT
16 0CTGAGCCATA
17 0TGAGCCATAG
18 0GAGCCATAGG
19 0AGCCATAGGC
20 0GCCATAGGCT
21 0CCATAGGCTA
22 0ACTGACCTGA
23 0CTGACCTGAC
24 0TGACCTGAC-
25 0GACCTGAC--
26 0ACCTGAC---
27 0CCTGAC----
28 0CTGAC----A
29 0TGAC----AG
30 0GAC----AGT
31 0AC----AGTC
32 0C----AGTCA
33 0----AGTCAG
34 0---AGTCAGA
35 0--AGTCAGAG
36 0-AGTCAGAGG
37 0AGTCAGAGGA
38 0GTCAGAGGAA
39 0TCAGAGGAAA
40 0CAGAGGAAAG
41 0AGAGGAAAGC
42 0GAGGAAAGCA
43 0AGGAAAGCAG
44 0GGAAAGCAGG
45 0GAAAGCAGGG
46 0AAAGCAGGGC
47 0AAGCAGGGCA
48 0AGCAGGGCAT
49 0GCAGGGCATA
50 0CAGGGCATAG
51 0AGGGCATAGA
52 0GGGCATAGAG
53 0GGCATAGAGA
54 0GCATAGAGAG
55 0CATAGAGAGG
56 0ATAGAGAGGT
57 0TAGAGAGGTT
58 0AGAGAGGTTT
59 0GAGAGGTTTG
60 0AGAGGTTTGA
61 0GAGGTTTGAA
62 0AGGTTTGAAC
63 0GGTTTGAACA
64 0GTTTGAACAC
65 0TTTGAACACT
66 0TTGAACACTT
67 0T